# Overview

## Coroutines
Use them to perform asynchronous tasks

## Tasks
Use them to perform multiple co-routines concurrently

## Task Groups

Use them to avoid explicit code required to run tasks and wait for them to finish

## Timeout and WaitFor

Use them to put a threshold wait time for any long running tasks

## Using Coroutine as Runners

In [4]:
import asyncio
async def do_async_work():
    print("Starting to do async work...")
    await asyncio.sleep(2)
    print("Finished the async work after two seconds")

await do_async_work()

Starting to do async work...
Finished the async work after two seconds


## Using Tasks

In [7]:
async def do_some_io_task(about: str, task_dur: int):
    print(f"Started to perform the task: {about}")
    await asyncio.sleep(task_dur)
    print(f"Finished the task: {about}")

task1 = asyncio.create_task(do_some_io_task("fecth water from well", 5))
task2 = asyncio.create_task(do_some_io_task("get vegitables from the market", 2))

await task1
await task2

task3 = asyncio.create_task(do_some_io_task("select a nice movie to watch", 7))

await task3



Started to perform the task: fecth water from well
Started to perform the task: get vegitables from the market
Finished the task: get vegitables from the market
Finished the task: fecth water from well
Started to perform the task: select a nice movie to watch
Finished the task: select a nice movie to watch


# Using Task Group

In [28]:
chop_list = [
    ("brinjal", 2),
    ("carrot", 3),
    ("onion", 12),
    ("tomato", 5)
]

async def do_chop_work(vegitable: str, cutting_dur: int):
    print(f"starting to cut {vegitable}")
    await asyncio.sleep(cutting_dur)
    return f"finished chopping {vegitable}"

tasks = [do_chop_work(name, dur) for name, dur in chop_list]

In [24]:
results = await asyncio.gather(*tasks)

print("\n".join(results))

starting to cut brinjal
starting to cut carrot
starting to cut onion
starting to cut tomato
finished chopping brinjal
finished chopping carrot
finished chopping onion
finished chopping tomato


# asyncio.gather Vs asyncio.wait

`asyncio.gather` and `asyncio.wait` are both functions in Python's asyncio library that deal with scheduling and controlling multiple tasks at once. While they have several similarities, there are also key differences:

1. **Return Values**:
   - `asyncio.gather`: returns the results of all tasks as a single list in the order they were passed in, regardless of the order in which they completed.
   - `asyncio.wait`: returns a pair of sets. The first set contains the futures that completed (either because they finished or because an error occurred) and the second includes the ones that didn't.

2. **Cancelling**:
   - `asyncio.gather`: if the `gather` call is cancelled, all subtasks will be cancelled.
   - `asyncio.wait`: if the `wait` is cancelled, it will cancel all provided tasks that are still running and immediately return.

3. **Error Handling**:
   - `asyncio.gather`: raises the first exception that was raised by one of the tasks. If `return_exceptions=True` is passed to `gather()`, exceptions are treated the same as successful results, and gathered in the result list.
   - `asyncio.wait`: does not raise exceptions from the tasks. You need to check each Future's `exception()` method manually.
   
In summary, the choice between them depends on your use case. If you want to simply run multiple tasks concurrently and gather their results, `asyncio.gather` is a good choice. If you need more control over which tasks have completed and which haven't, or need to handle exceptions manually, `asyncio.wait` may be the better choice.

# Consider the scenario where you want to execute multiple tasks and if some task is a long running task you want to timeout and return but still have other tasks finish their work

In [32]:
import asyncio

chop_list = [
    ("brinjal", 2),
    ("carrot", 3),
    ("onion", 12),
    ("tomato", 5)
]

MAX_TIME_OUT_FOR_TASK = 10

async def do_chop_work(vegitable: str, cutting_dur: int):
    print(f"starting to cut {vegitable}")
    await asyncio.sleep(cutting_dur)
    return f"finished chopping {vegitable}"

# create a list to store all the tasks
tasks = [asyncio.wait_for(do_chop_work(vegetable, duration), timeout=MAX_TIME_OUT_FOR_TASK)
            for vegetable, duration in chop_list]

# gather the results, if a task is cancelled due to a TimeoutError, store the exception instead
results = await asyncio.gather(*tasks, return_exceptions=True)

# process the results
for task, result in zip(chop_list, results):
    if isinstance(result, asyncio.TimeoutError):
        print(f"Task {task[0]} was timeout because it ran too long.")
    elif isinstance(result, asyncio.CancelledError):
        print(f"Task {task[0]} was cancelled because it ran too long.")
    elif isinstance(result, Exception):
        print(f"Task {task[0]} raised an exception: {str(result)}")
    else:
        print(f"Task {task[0]} result: {result}")

starting to cut brinjal
starting to cut carrot
starting to cut onion
starting to cut tomato
Task brinjal result: finished chopping brinjal
Task carrot result: finished chopping carrot
Task onion was timeout because it ran too long.
Task tomato result: finished chopping tomato
